In [1]:
import pandas as pd

# Load the simulated price paths
simulated_data = pd.read_csv('simulated_price_paths.csv')

# Display the first few rows of the data to verify
print(simulated_data.head())


   Time_Step            0            1            2            3            4  \
0          0  5808.000000  5808.000000  5808.000000  5808.000000  5808.000000   
1          1  5808.218858  5807.939938  5808.285177  5808.669705  5807.897819   
2          2  5808.205022  5807.719385  5808.210128  5808.984369  5808.459810   
3          3  5808.591958  5807.737322  5807.810057  5808.781855  5808.611094   
4          4  5808.297703  5807.424477  5808.436855  5809.601643  5808.203722   

             5            6            7            8  ...         7990  \
0  5808.000000  5808.000000  5808.000000  5808.000000  ...  5808.000000   
1  5807.897826  5808.694387  5808.337779  5807.794456  ...  5808.531854   
2  5808.149087  5808.739745  5808.996532  5807.657719  ...  5807.986213   
3  5808.948599  5808.527786  5809.350779  5807.514300  ...  5808.800064   
4  5808.655372  5809.030381  5809.585860  5807.812077  ...  5808.666899   

          7991         7992         7993         7994         

In [2]:
import numpy as np

# Define parameters
strike_prices = [5620, 5640, 5660, 5680, 5700, 5720, 5740, 5760, 5780, 5800]
risk_free_rate = 0.05  # Annualized risk-free rate
dt = 1 / 252  # Time step (1 trading day)

# Extract simulated prices (drop 'Time_Step')
simulated_prices = simulated_data.drop(columns=['Time_Step']).values

# Dictionaries to store option values for calls and puts
call_values = {}
put_values = {}

for strike in strike_prices:
    # Payoff at expiry
    call_payoff = np.maximum(simulated_prices - strike, 0)
    put_payoff = np.maximum(strike - simulated_prices, 0)

    # Initialize option value arrays (same shape as simulated_prices)
    call_value = np.copy(call_payoff)
    put_value = np.copy(put_payoff)

    # Backward induction: Start from the last time step and move backward
    for t in range(simulated_prices.shape[0] - 2, -1, -1):
        # Discounted continuation value
        call_continuation = np.exp(-risk_free_rate * dt) * call_value[t + 1]
        put_continuation = np.exp(-risk_free_rate * dt) * put_value[t + 1]

        # Compare immediate exercise payoff vs continuation value
        call_value[t] = np.maximum(call_payoff[t], call_continuation)
        put_value[t] = np.maximum(put_payoff[t], put_continuation)

    # Store results
    call_values[strike] = call_value
    put_values[strike] = put_value

# Convert to DataFrame for saving and inspection
call_values_df = pd.concat({f"Call_{strike}": pd.DataFrame(values) for strike, values in call_values.items()}, axis=1)
put_values_df = pd.concat({f"Put_{strike}": pd.DataFrame(values) for strike, values in put_values.items()}, axis=1)

# Save results to CSV
call_values_df.to_csv('american_call_values.csv', index=False)
put_values_df.to_csv('american_put_values.csv', index=False)

print("American option values (calls and puts) saved to CSV files.")


American option values (calls and puts) saved to CSV files.


In [3]:
# Load the saved CSV files
call_values_df = pd.read_csv('american_call_values.csv')
put_values_df = pd.read_csv('american_put_values.csv')

# Display the first few rows of each DataFrame
print("American Call Option Values:")
print(call_values_df.head())

print("\nAmerican Put Option Values:")
print(put_values_df.head())


American Call Option Values:
    Call_5620  Call_5620.1  Call_5620.2  Call_5620.3  Call_5620.4  \
0    0.000000     1.000000     2.000000     3.000000     4.000000   
1  212.630810   202.985774   192.074448   189.451225   237.824004   
2  212.673003   203.026053   192.112562   189.488819   237.871196   
3  212.715204   203.066340   192.150683   189.526419   237.918398   
4  212.757414   203.106635   192.188812   189.564028   237.965609   

   Call_5620.5  Call_5620.6  Call_5620.7  Call_5620.8  Call_5620.9  ...  \
0     5.000000     6.000000     7.000000     8.000000     9.000000  ...   
1   207.747899   199.491700   240.136837   196.036245   209.311335  ...   
2   207.789123   199.531285   240.184488   196.075145   209.352869  ...   
3   207.830355   199.570879   240.232148   196.114052   209.394411  ...   
4   207.871595   199.610480   240.279818   196.152968   209.435962  ...   

   Call_5800.7990  Call_5800.7991  Call_5800.7992  Call_5800.7993  \
0     7990.000000     7991.000000   